In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"

import util_funcs
from importlib import reload
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
import predictSeizureConvExp as psce
import keras_models.dataGen as dg
from addict import Dict
reload(psce)
from keras.utils import multi_gpu_model
import keras.optimizers
import time
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, log_loss

Using TensorFlow backend.


In [2]:
config = Dict(psce.standardized_ensemble())
config.keys()

dict_keys(['use_random_ensemble', 'precached_pkl', 'precached_test_pkl', 'ensemble_sample_info_path', 'max_num_samples', 'use_standard_scaler', 'use_filtering'])

In [3]:
precached_pkl = "/n/scratch2/ms994/standardized_simple_ensemble_train_data_seizure.pkl"

In [3]:
trainDataGen = psce.get_data_generator("train", config.precached_pkl)

In [4]:
testDataGen = psce.get_data_generator("dev_test", config.precached_test_pkl, is_test=True)

In [6]:
trainDataGen, validDataGen = trainDataGen

In [30]:
len(testDataGen)

1093

In [7]:
testX = np.stack([datum[0] for datum in testDataGen.dataset])
testY = np.stack([datum[1] for datum in testDataGen.dataset])

In [39]:
testX.shape, testY.shape

((34998, 500, 21), (34998,))

In [86]:
seizureX = np.vstack([testX[testY == 1], testX[0:2500]])
seizureX.shape

(5018, 500, 21)

In [87]:
seizureY = np.hstack([testY[testY == 1], testY[0:2500]])
seizureY.shape, seizureY.mean()

((5018,), 0.5245117576723795)

In [ ]:
seizy

In [8]:
import keras_models.vanPutten as vp

In [9]:
reload(vp)
model = vp.inception_like((21,500,1), num_layers=4, num_filters=10)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 21, 500, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 20, 499, 10)  50          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 19, 498, 10)  100         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 18, 497, 10)  170         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_13 

In [11]:
model = multi_gpu_model(model, 4)

In [12]:
adam = keras.optimizers.Adam(lr=0.001)
model.compile(adam, loss="categorical_crossentropy")

In [13]:
from keras_models import util

In [15]:
validData = validDataGen.dataset

In [18]:
validDataX = np.stack([datum[0] for datum in validData])
validDataY = np.stack([datum[1] for datum in validData])

(500, 21)

In [24]:
y_true_a = []
for i in range(len(validDataGen)):
    _, y_true_add = validDataGen[i]
    y_true.append(y_true_add)

In [29]:
np.concatenate(y_true)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [25]:
y_true == y_true_a

False

In [20]:
y_pred = model.predict_generator(validDataGen)

In [21]:
y_pred.shape

(17952, 2)

<module 'keras_models.util' from '/home/ms994/dbmi_eeg_clustering/keras_models/util.py'>

In [54]:
from keras_models import util as util_run
reload(util_run)
history = util_run.model_run(model=model, patience=10, class_weights=[1,30], model_name="yolo1.h5", epochs=1, trainDataGen=trainDataGen, validDataGen=validDataGen, )

Progress: 1/3267
Epoch: 0, Step: 1/256, Loss: 1.1920930376163597e-07
Progress: 2/3267
Epoch: 0, Step: 2/256, Loss: 0.11542464047670364
Progress: 3/3267
Epoch: 0, Step: 3/256, Loss: 1.5646222095710982e-07
Progress: 4/3267
Epoch: 0, Step: 4/256, Loss: 1.5127218961715698
Progress: 5/3267
Epoch: 0, Step: 5/256, Loss: 1.528052806854248
Epoch 1/1: loss: 0.6312398910522461
Progress: 1/3267
Epoch: 1, Step: 1/256, Loss: 1.1920930376163597e-07
Progress: 2/3267
Epoch: 1, Step: 2/256, Loss: 1.5110714435577393
Progress: 3/3267
Epoch: 1, Step: 3/256, Loss: 1.0073812007904053
Progress: 4/3267
Epoch: 1, Step: 4/256, Loss: 5.476194928633049e-07
Progress: 5/3267
Epoch: 1, Step: 5/256, Loss: 1.1230889558792114
Epoch 2/1: loss: 0.728308379650116
Progress: 1/3267
Epoch: 2, Step: 1/256, Loss: 0.000744776742067188
1.2294352054595947


In [56]:
len(history.history.loss)

2

In [102]:
trainDataGen.batch_size = 1280

In [103]:
start = time.time()
lossHist = []
callback_check = 10
epochs=1
# seizureX = seizureX.reshape((*seizureX.shape, 1)).transpose((0,2,1,3))
num_steps = len(trainDataGen)
# num_steps = 10
validX = []
validY = []

In [106]:
callback_check = 5
start = time.time()
class_weight = np.array([1,20])
update_amount = 0.9
for i in range((epochs)):
    for j in range(num_steps):
        loss = model.train_on_batch(trainDataGen[j][0], trainDataGen[j][1], class_weight=class_weight)
        lossHist.append(loss)
        print("Epoch: {}, Step: {}/{}, Loss: {}".format(i,j, num_steps, loss))
        if j % callback_check == 0:
            valid_index = np.random.choice(list(range(len(seizureX))), 1000)
            validX = seizureX[valid_index]
            validY = seizureY[valid_index]
            y_pred = model.predict(validX)
            y_pred = y_pred.argmax(1)
            if y_pred.mean() == int(y_pred.mean()):
                problem_class = int(y_pred.mean())
                class_weight = class_weight / update_amount
                class_weight[problem_class] = class_weight[problem_class] * update_amount ** 2
                print("predicted all {}".format(problem_class))
                print("Updating Class Weight to: {}".format(class_weight))

print(time.time()-start)

Epoch: 0, Step: 0/95, Loss: 0.2845654785633087
predicted all 0
Updating Class Weight to: [ 0.9        22.22222222]
Epoch: 0, Step: 1/95, Loss: 0.26884907484054565
Epoch: 0, Step: 2/95, Loss: 0.3087906539440155
Epoch: 0, Step: 3/95, Loss: 0.30356794595718384
Epoch: 0, Step: 4/95, Loss: 0.24307191371917725
Epoch: 0, Step: 5/95, Loss: 0.29212602972984314
predicted all 0
Updating Class Weight to: [ 0.81       24.69135802]
Epoch: 0, Step: 6/95, Loss: 0.2752738893032074
Epoch: 0, Step: 7/95, Loss: 0.255071759223938
Epoch: 0, Step: 8/95, Loss: 0.25660499930381775
Epoch: 0, Step: 9/95, Loss: 0.2733497619628906
Epoch: 0, Step: 10/95, Loss: 0.19150973856449127
predicted all 0
Updating Class Weight to: [ 0.729      27.43484225]
Epoch: 0, Step: 11/95, Loss: 0.2439744919538498
Epoch: 0, Step: 12/95, Loss: 0.247213214635849
Epoch: 0, Step: 13/95, Loss: 0.2706335186958313
Epoch: 0, Step: 14/95, Loss: 0.3031953275203705
Epoch: 0, Step: 15/95, Loss: 0.28288760781288147
predicted all 0
Updating Class We

In [107]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [84]:
class_weight = class_weight / 0.75

In [78]:
1.0 == int(1)

True

In [109]:
y_pred = model.predict(seizureX)

In [114]:
confusion_matrix(seizureY, y_pred.argmax(1),)

array([[2386,    0],
       [2629,    3]])

AttributeError: 'numpy.ndarray' object has no attribute 'get_shape'